In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
cwd = os.getcwd()
import librosa
import librosa.display
import numpy as np
import pandas as pd
import natsort
import matplotlib.pyplot as plt
import scipy


In [3]:
wavDir = '/content/drive/MyDrive/Multimedia_Systems/Lab-2/dataset/ff1010bird_wav/wav/'
# read audio files

listofwavs = os.listdir(wavDir)
print(len(listofwavs))

sortwavdir = natsort.natsorted(listofwavs, reverse=False)
print(sortwavdir)

7690
['99.wav', '100.wav', '104.wav', '156.wav', '345.wav', '347.wav', '349.wav', '369.wav', '370.wav', '371.wav', '376.wav', '377.wav', '378.wav', '518.wav', '1042.wav', '1043.wav', '1044.wav', '1045.wav', '1046.wav', '1050.wav', '1051.wav', '1052.wav', '1053.wav', '1054.wav', '1055.wav', '1056.wav', '1748.wav', '1931.wav', '1932.wav', '1933.wav', '1934.wav', '1945.wav', '2151.wav', '2155.wav', '2157.wav', '2421.wav', '2432.wav', '2435.wav', '2519.wav', '2520.wav', '2521.wav', '2522.wav', '2523.wav', '2524.wav', '2525.wav', '2526.wav', '2527.wav', '2528.wav', '2529.wav', '2530.wav', '2531.wav', '2532.wav', '2533.wav', '2534.wav', '2535.wav', '2536.wav', '2537.wav', '2689.wav', '3093.wav', '3126.wav', '3177.wav', '3178.wav', '3179.wav', '3180.wav', '3182.wav', '3183.wav', '3184.wav', '3185.wav', '3186.wav', '3187.wav', '3188.wav', '3189.wav', '3190.wav', '3191.wav', '3202.wav', '5204.wav', '5560.wav', '5929.wav', '5996.wav', '5997.wav', '6108.wav', '7126.wav', '7133.wav', '7151.wav', '

In [ ]:
sampleAll = []
import librosa
for i in range(len(sortwavdir)):
  path2Audio = wavDir + sortwavdir[i]
  audio,sr = librosa.load(path2Audio)
  fourSecSample= audio[4*sr:8*sr] #take 4 second from each sample
  #print(sortwavdir[i], sr, len(audio)/sr)
  sampleAll.append(fourSecSample)

In [ ]:
sampleAll = np.asarray(sampleAll)
print(type(sampleAll),np.shape(audio))

In [ ]:
# Load Label

labeldir = '/content/drive/MyDrive/Multimedia_Systems/Lab-2/dataset/ff1010bird_wav/ff1010bird_metadata.csv'

df = pd.read_csv(labeldir, header = None, skiprows=1)
data = df.values

# Sort the labels
sortedArr = data[data[:,0].argsort()]

# Lets separate the file name and labels

sortedLabel = sortedArr[:,1]
sortedfilename  = sortedArr[:,0]

print(sortedfilename[0:])
label = sortedLabel[0:]
print(label)
  

In [ ]:
def Feature_Extraction(samples):
  allFeature=[]
  for frame in range(len(samples)):
    # print(frame)
    OneSample=samples[frame,:]
    ###MFCC
    feature_mfcc=librosa.feature.mfcc(y=OneSample, sr=sr,n_mfcc=13,  hop_length=len(OneSample)+1)  
    ### energy
    #feature_eng=np.sum(np.square(OneSample)).reshape(-1,1)
    ####ZCR
    feature_zcr=librosa.feature.zero_crossing_rate(OneSample,hop_length=len(OneSample)+1)  
    ###Spectral Roll_off
    feature_spectral=librosa.feature.spectral_rolloff(OneSample, sr=sr , hop_length=len(OneSample)+1)

    featuresOneFrame=np.concatenate((feature_mfcc,feature_zcr,feature_spectral),axis = 0)  
    allFeature.append(featuresOneFrame)
  return allFeature

In [ ]:
#concatenate the extracted features with existing dataset
features = Feature_Extraction(sampleAll)
features = np.asarray(features)
features = np.reshape(features,(features.shape[0],features.shape[1]))
features.shape

In [ ]:
#Split the dataset into train and test (80:20)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, train_size = 0.8, 
                                                    random_state = 101, shuffle = True)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

my_model = Sequential()
my_model.add(Input(shape=(15,)))
my_model.add(Dense(12, activation='relu',name='d1'))
my_model.add(Dense(8, activation='relu',name='d2'))
my_model.add(Dense(4, activation='relu',name='d3'))
# my_model.add(Dense(3, activation='softmax',name='out'))##if 3-class classification
my_model.add(Dense(1, activation='sigmoid',name='out')) ##if 2-class classification

# summarize layers
print(my_model.summary())
# plot graph
plot_model(my_model, to_file='model_graph.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Multilayer Perceptron
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
#visible = Input(shape=(sampleAll.shape[1],),name='modelInput')
visible = Input(15,name='modelInput')
hidden1 = Dense(8, activation='relu',name='hid1')(visible)
hidden2 = Dense(4, activation='relu',name='hid2')(hidden1)
hidden3 = Dense(2, activation='relu',name='hid3')(hidden2)

# output = Dense(3, activation='softmax',name='modelOutput')(hidden3)##if 3-class classification
output = Dense(1, activation='sigmoid',name='modelOutput')(hidden3)##if 2-class classification

fun_model = Model(inputs=visible, outputs=output)
# summarize layers
print(fun_model.summary())
# plot graph
plot_model(fun_model, to_file='model_graph.png', show_shapes=True, show_layer_names=True)


In [ ]:
# compile the keras model
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam

In [ ]:
# my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])
fun_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy','mse','mae'])

In [ ]:
# fit the keras model on the dataset
my_history=fun_model.fit(X_train, y_train, epochs=10, batch_size=4,verbose=1, validation_split=0.1)

In [ ]:
def plot(title, a, b, c, d, e, f):

  plt.subplot(211)
  plt.title(title)
  plt.plot(my_history.history[a],label = b)
  plt.plot(my_history.history[c],label = d)
  plt.xlabel(e)
  plt.ylabel(f)
  plt.legend()
  plt.show()

In [ ]:
plot('Loss_plot', 'loss', 'train', 'val_loss', 'val', 'epochs', 'loss')

In [ ]:
plot('mse_plot', 'mse', 'train', 'val_mse', 'val', 'epochs', 'mse')

In [ ]:
plot('acc_plot', 'accuracy', 'train_accuracy', 'val_accuracy', 'val', 'epochs', 'accuracy')

In [ ]:
import keras

#### make class predictions with the model
y_pred = fun_model.predict(X_test)
#########direct evaluaion
score = fun_model.evaluate(X_test, y_test)
print(score[1])

In [ ]:
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
# save model and architecture to single file
fun_model.save("fun_model.h5")
print("Saved model to disk")
 
# later...
 
from keras.models import load_model
 
# load model
fun_model = load_model('fun_model.h5')

In [ ]:
score = fun_model.evaluate(X_test, y_test, verbose=1)
print("%s: %.2f%%" % (fun_model.metrics_names[1], score[1]*100))

#Another Variant of NN

In [ ]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

my_model = Sequential()
my_model.add(Input(shape=(15,)))
my_model.add(Dense(12, activation='relu',name='d1'))
my_model.add(Dense(8, activation='relu',name='d2'))
my_model.add(Dense(4, activation='relu',name='d3'))
#my_model.add(Dense(3, activation='softmax',name='out'))##if 3-class classification
my_model.add(Dense(3, activation='sigmoid',name='out')) ##if 2-class classification

# summarize layers
print(my_model.summary())
# plot graph
plot_model(my_model, to_file='model_graph.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Multilayer Perceptron
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
#visible = Input(shape=(sampleAll.shape[1],),name='modelInput')
visible = Input(15,name='modelInput')
hidden1 = Dense(8, activation='relu',name='hid1')(visible)
hidden2 = Dense(4, activation='relu',name='hid2')(hidden1)
hidden3 = Dense(2, activation='relu',name='hid3')(hidden2)
#output = Dense(1, activation='softmax',name='modelOutput')(visible) ##if 3-class classification

output = Dense(1, activation='sigmoid',name='modelOutput')(hidden3)##if 2-class classification

new_model = Model(inputs=visible, outputs=output)
# summarize layers
print(new_model.summary())
# plot graph
plot_model(new_model, to_file='model_graph.png', show_shapes=True, show_layer_names=True)

In [ ]:
# compile the keras model
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam

In [ ]:
#my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse','mae'])
new_model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.001), metrics=['accuracy','mse','mae'])

In [ ]:
# fit the keras model on the dataset
my_history=new_model.fit(X_train, y_train, epochs=10, batch_size=4,verbose=3, validation_split=0.1)

In [ ]:
def plot(title, a, b, c, d, e, f):

  plt.subplot(211)
  plt.title(title)
  plt.plot(my_history.history[a],label = b)
  plt.plot(my_history.history[c],label = d)
  plt.xlabel(e)
  plt.ylabel(f)
  plt.legend()
  plt.show()

In [ ]:
plot('Loss_plot', 'loss', 'train', 'val_loss', 'val', 'epochs', 'loss')

In [ ]:
plot('mse_plot', 'mse', 'train', 'val_mse', 'val', 'epochs', 'mse')

In [ ]:
plot('acc_plot', 'accuracy', 'train_accuracy', 'val_accuracy', 'val', 'epochs', 'accuracy')

In [ ]:
import keras

#### make class predictions with the model
y_pred = new_model.predict(X_test)
#########direct evaluaion
score = new_model.evaluate(X_test, y_test)
print(score[1])

In [ ]:
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
score = new_model.evaluate(X_test, y_test, verbose=1)
print("%s: %.2f%%" % (new_model.metrics_names[1], score[1]*100))